# Retrieving Image Collection Metadata

In [1]:
##Import required Google Earth Engine python packages and check if they work in python environment
import ee
ee.Initialize()
import geemap
import pandas as pd

In [2]:
#import the map module that allows for attaching images to an interactive map
Map = geemap.Map()

## Import Google Earth Engine Feature Collection Data

In [3]:
#Import the study area boundary from the Google Earth Engine Server
#https://code.earthengine.google.com/?asset=users/pjf927/Chile_Project/Study_Area_Fitted
#Good youtube tutorial on how to import your own shapefile: https://www.youtube.com/watch?v=AmaaEFbBtfQ&ab_channel=INSGIS
#Call in the study area boundary from a vector file saved into Google Earth Engine 
Study_Area = ee.FeatureCollection("users/pjf927/Chile_Project/Study_Area_Fitted")
#Some function require geometry values to clip features
Study_Area_Geom = Study_Area.geometry() 

#Call in the big study area boundary from a vector file saved into Google Earth Engine 
Big_Study_Area = ee.FeatureCollection("users/pjf927/Chile_Project/Study_Area_Big")
#Some function require geometry values to clip features
Big_Area_Geom = Big_Study_Area.geometry() 

#Add study boundary to the map
Map.addLayer(Study_Area, name = "Study_Area")
Map.addLayer(Big_Study_Area, name = "Big_Study_Area")

#Display the images centered at the whole coverage of the vector file
Map.centerObject(Big_Study_Area, 13);
Map

Map(center=[-33.47385665572837, -70.83305894881653], controls=(WidgetControl(options=['position', 'transparent…

## Import Google Earth Engine Image Collection Data

In [4]:
#Call in Landsat 9 Level 2, Collection 2, Tier 1 dataset 
LS9_SR = (ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(Study_Area) #Filter only swath grids that cover the TN River Boundary
    #.filterDate("2022-10-01", "2022-12-31") #Filter Dates of Landsat acquisitions
    .sort('system:time_start') #Sort collection by acquisition time
)

#Get a count of how many  images are in the collection
LS9_count = LS9_SR.size().getInfo()
print("Landsat 9 Images: ", LS9_count)

Landsat 9 Images:  27


### Get date and index parameters from the metadata's information

In [5]:
#Create a function that creates a new dictionary in the image collection called 'Date' and converts the 'system:time_start' list to a "YYYY-MM-dd" format
LS9_SR = LS9_SR.map(
    lambda img: img.set({"DATE": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")})
)

#Create a list of dates and print them out
Dates = LS9_SR.aggregate_array("DATE").getInfo()
print("Dates in Imagecollection: \n", Dates)

Index = LS9_SR.aggregate_array('system:index').getInfo()
print("Index Names in Imagecollection: \n", Index)

Dates in Imagecollection: 
 ['2021-12-06', '2021-12-22', '2022-01-07', '2022-01-23', '2022-02-08', '2022-02-24', '2022-03-28', '2022-04-13', '2022-04-29', '2022-05-15', '2022-05-31', '2022-06-16', '2022-07-02', '2022-07-18', '2022-08-03', '2022-08-19', '2022-09-04', '2022-09-20', '2022-10-06', '2022-10-22', '2022-11-07', '2022-11-23', '2022-12-09', '2022-12-25', '2023-01-10', '2023-01-26', '2023-02-11']
Index Names in Imagecollection: 
 ['LC09_233083_20211206', 'LC09_233083_20211222', 'LC09_233083_20220107', 'LC09_233083_20220123', 'LC09_233083_20220208', 'LC09_233083_20220224', 'LC09_233083_20220328', 'LC09_233083_20220413', 'LC09_233083_20220429', 'LC09_233083_20220515', 'LC09_233083_20220531', 'LC09_233083_20220616', 'LC09_233083_20220702', 'LC09_233083_20220718', 'LC09_233083_20220803', 'LC09_233083_20220819', 'LC09_233083_20220904', 'LC09_233083_20220920', 'LC09_233083_20221006', 'LC09_233083_20221022', 'LC09_233083_20221107', 'LC09_233083_20221123', 'LC09_233083_20221209', 'LC09_

### Get all metadata from a single image out of the image collection

In [6]:
#Can't display the whole image collection's metadata, you have to do a single one as a single image data type
First_LS9 = LS9_SR.first()
geemap.image_props(First_LS9).getInfo()

{'ALGORITHM_SOURCE_SURFACE_REFLECTANCE': 'LaSRC_1.5.0',
 'ALGORITHM_SOURCE_SURFACE_TEMPERATURE': 'st_1.3.0',
 'CLOUD_COVER': 30.12,
 'CLOUD_COVER_LAND': 25.08,
 'COLLECTION_CATEGORY': 'T1',
 'COLLECTION_NUMBER': 2,
 'DATA_SOURCE_AIR_TEMPERATURE': 'MODIS',
 'DATA_SOURCE_ELEVATION': 'GLS2000',
 'DATA_SOURCE_OZONE': 'MODIS',
 'DATA_SOURCE_PRESSURE': 'Calculated',
 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT',
 'DATA_SOURCE_WATER_VAPOR': 'MODIS',
 'DATE': '2021-12-06',
 'DATE_ACQUIRED': '2021-12-06',
 'DATE_PRODUCT_GENERATED': 1642677395000,
 'DATUM': 'WGS84',
 'EARTH_SUN_DISTANCE': 0.9853215,
 'ELLIPSOID': 'WGS84',
 'GEOMETRIC_RMSE_MODEL': 4.322,
 'GEOMETRIC_RMSE_MODEL_X': 3.134,
 'GEOMETRIC_RMSE_MODEL_Y': 2.975,
 'GEOMETRIC_RMSE_VERIFY': 3.147,
 'GRID_CELL_SIZE_REFLECTIVE': 30,
 'GRID_CELL_SIZE_THERMAL': 30,
 'GROUND_CONTROL_POINTS_MODEL': 524,
 'GROUND_CONTROL_POINTS_VERIFY': 185,
 'GROUND_CONTROL_POINTS_VERSION': 5,
 'IMAGE_DATE': '2021-12-06',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS

### Get metadata for a specific date out of the image collection

In [7]:
#If you want a particular date for an image enter the index number
LS9_2022_12_09 = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_233083_20221209')
geemap.image_props(LS9_2022_12_09).getInfo()

{'ALGORITHM_SOURCE_SURFACE_REFLECTANCE': 'LaSRC_1.5.0',
 'ALGORITHM_SOURCE_SURFACE_TEMPERATURE': 'st_1.3.0',
 'CLOUD_COVER': 3.67,
 'CLOUD_COVER_LAND': 0.75,
 'COLLECTION_CATEGORY': 'T1',
 'COLLECTION_NUMBER': 2,
 'DATA_SOURCE_AIR_TEMPERATURE': 'MODIS',
 'DATA_SOURCE_ELEVATION': 'GLS2000',
 'DATA_SOURCE_OZONE': 'MODIS',
 'DATA_SOURCE_PRESSURE': 'Calculated',
 'DATA_SOURCE_REANALYSIS': 'GEOS-5 FP-IT',
 'DATA_SOURCE_WATER_VAPOR': 'MODIS',
 'DATE_ACQUIRED': '2022-12-09',
 'DATE_PRODUCT_GENERATED': 1670743431000,
 'DATUM': 'WGS84',
 'EARTH_SUN_DISTANCE': 0.9849237,
 'ELLIPSOID': 'WGS84',
 'GEOMETRIC_RMSE_MODEL': 5.117,
 'GEOMETRIC_RMSE_MODEL_X': 3.532,
 'GEOMETRIC_RMSE_MODEL_Y': 3.702,
 'GEOMETRIC_RMSE_VERIFY': 2.891,
 'GRID_CELL_SIZE_REFLECTIVE': 30,
 'GRID_CELL_SIZE_THERMAL': 30,
 'GROUND_CONTROL_POINTS_MODEL': 1139,
 'GROUND_CONTROL_POINTS_VERIFY': 342,
 'GROUND_CONTROL_POINTS_VERSION': 5,
 'IMAGE_DATE': '2022-12-09',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'L1_DATE_PRODUCT_

### Create a  visualization parameter for true color image

In [8]:
#Visualization parameters to be added to a graph
vis_params_No_Scale = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 1,
  'max': 65455,
}

### Create a  visualization parameter for thermal image

In [9]:
#Visualization parameters to be added to a graph
vis_params_Therm = {
  'bands': ['ST_B10'],
  'min': 0,
  'max': 65535,
}

### Display the single images for the that were created

In [ ]:
#Display in true color the first image and the 2022-12-09 image
Map.addLayer(First_LS9, vis_params_No_Scale, 'First TC Image')
Map.addLayer(LS9_2022_12_09, vis_params_No_Scale, '12-09 TC Image')

#Display in thermal the first image and the 2022-12-09 image
Map.addLayer(First_LS9, vis_params_Therm, 'First Therm Image')
Map.addLayer(LS9_2022_12_09, vis_params_Therm, '12-09 Therm Image')

#Add study boundary to the map (I did this again because they wouldn't be in the right order)
Map.addLayer(Study_Area, name = "Study_Area")
Map.addLayer(Big_Study_Area, name = "Big_Study_Area")
Map

## Your Task: Copy and paste metadata of  image 2022-08-03 into a text document and send to me

In [ ]:
#If you want a particular date for an image enter the index number
LS9_2022_08_03 = ee.Image('')
geemap.image_props(LS9_2022_08_03).getInfo()

## Create pandas dataframe from all the metadata in the image collection and convert data into a csv file 

In [10]:
#Create a list of sensor acquisitions and print them out
Sensor = LS9_SR.aggregate_array("SPACECRAFT_ID").getInfo()
print(Sensor)

['LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9', 'LANDSAT_9']


In [14]:
#Create a list of swath row acquisitions and print them out
Row = LS9_SR.aggregate_array("WRS_ROW").getInfo()
print(Row)

[83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83]


In [15]:
#Create a list of swath row acquisitions and print them out
Path = LS9_SR.aggregate_array("WRS_PATH").getInfo()
print(Path)

[233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233, 233]


In [12]:
#Create a list of cloud percentages and print them out
CLD_Percent = LS9_SR.aggregate_array('CLOUD_COVER').getInfo()
print(CLD_Percent)

[30.12, 16.91, 4.63, 0.92, 6.14, 20.78, 5.75, 0.54, 5.28, 5.61, 27.59, 12.03, 40.85, 30.22, 30.85, 1.86, 14.51, 54.61, 9.3, 13.36, 17.03, 14.07, 3.67, 11.61, 45.13, 2.89, 3.98]


In [16]:
#Save each list as a pandas dataframe
Acquisition_Data = pd.DataFrame(list(zip(Dates, Sensor, Row, Path, CLD_Percent)), 
                                columns=['Date','Sensor', 'Swath_Row', 'Swath_Path', 'Cloud_Percent'])

print(Acquisition_Data)

          Date     Sensor  Swath_Row  Swath_Path  Cloud_Percent
0   2021-12-06  LANDSAT_9         83         233          30.12
1   2021-12-22  LANDSAT_9         83         233          16.91
2   2022-01-07  LANDSAT_9         83         233           4.63
3   2022-01-23  LANDSAT_9         83         233           0.92
4   2022-02-08  LANDSAT_9         83         233           6.14
5   2022-02-24  LANDSAT_9         83         233          20.78
6   2022-03-28  LANDSAT_9         83         233           5.75
7   2022-04-13  LANDSAT_9         83         233           0.54
8   2022-04-29  LANDSAT_9         83         233           5.28
9   2022-05-15  LANDSAT_9         83         233           5.61
10  2022-05-31  LANDSAT_9         83         233          27.59
11  2022-06-16  LANDSAT_9         83         233          12.03
12  2022-07-02  LANDSAT_9         83         233          40.85
13  2022-07-18  LANDSAT_9         83         233          30.22
14  2022-08-03  LANDSAT_9         83    

In [17]:
#Save dataframe as a csv to a folder directory
pd.DataFrame(Acquisition_Data).to_csv(r'D:\Azad_Lab\Chile_Proj\Code\Patrick_Files\Data\csv\L9_SR_Metadata_Data.csv')

## Your Task: Create a new dataframe with max and min temperature and send csv to me

In [ ]:
#Create a list of minimum and maximaum temperature values in kelvin
Temp_Min = LS9_SR.aggregate_array('').getInfo()
print(Temp_Min)

Temp_Max = LS9_SR.aggregate_array('').getInfo()
print(Temp_Max)

In [ ]:
#Save each list as a pandas dataframe
Acquisition_Temp_Data = pd.DataFrame(list(zip(Dates, Sensor, Row, Path, CLD_Percent, Temp_Min , Temp_Max), 
                                columns=['Date','Sensor', 'Swath_Row', 'Swath_Row', 'Cloud_Percent', 'Minimum Temp (K)', 'Maximum Temp (K)'])            
                                     
print(Acquisition_Temp_Data)

In [ ]:
#Save dataframe as a csv to a folder directory
pd.DataFrame(Acquisition_Data).to_csv(r"D:\Azad_Lab\Chile_Proj\Code\Patrick_Files\Data\csv\L8_SR_Temp_Metadata_Data.csv")